# Trabajo Práctico Nro. 2
## Mineria de Datos - 2024.
#### Tec. Universitaria en Inteligencia Artificial - FCEIA (UNR).

### Integrantes:
 * Pace, Bruno. Legajo: P-5295/7.
 * Sancho Almenar, Mariano. Legajo: S-5778/9.

[Link al repositorio](https://github.com/bpace1/TP3-Mineria-De-Datos)

In [184]:
# Manejo de datos
import pandas as pd
import numpy as np

# Gráficos
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import seaborn as sns
import matplotlib.pyplot as plt

# Modealdo
from sklearn.preprocessing import StandardScaler
#from mlxtend.plotting import plot_decision_regions
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score 
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, recall_score, f1_score


# Tipado
from typing import Dict, Any, List

# Wanings
import warnings
warnings.filterwarnings("ignore")

# Path
import os


#### Lectura de dataset

Lectura del dataset de los datos del tiempo con la ayuda de la librería OS para poder trabajar sin problemas con las rutas del archivo.

In [148]:
PATH = os.getcwd()
DATA_PATH = os.path.join(PATH, 'data')

In [149]:
df: pd.DataFrame = pd.read_csv(os.path.join(DATA_PATH,'dxWeather.csv'))

# Pre - procesamiento de datos
- EDA.
- Transformación de datos.
- Visualización de datos.


Descripción del dataset: 

- **Temperatura (numérica)**: La temperatura en grados Celsius, que varía desde frío extremo hasta calor extremo.  
- **Humedad (numérica)**: El porcentaje de humedad, incluyendo valores superiores al 100% para introducir valores atípicos.  
- **Velocidad del Viento (numérica)**: La velocidad del viento en kilómetros por hora, con un rango que incluye valores irrealistamente altos.  
- **Precipitación (%) (numérica)**: El porcentaje de precipitación, incluyendo valores atípicos.  
- **Cobertura de Nubes (categórica)**: La descripción de la cobertura de nubes.  
- **Presión Atmosférica (numérica)**: La presión atmosférica en hPa, cubriendo un amplio rango.  
- **Índice UV (numérica)**: El índice UV, que indica la intensidad de la radiación ultravioleta.  
- **Estación (categórica)**: La estación del año en la que se registraron los datos.  
- **Visibilidad (km) (numérica)**: La visibilidad en kilómetros, incluyendo valores muy bajos o muy altos.  
- **Ubicación (categórica)**: El tipo de ubicación donde se registraron los datos.  
- **Tipo de Clima (categórica)**: La variable objetivo para la clasificación, que indica el tipo de clima.  


[Link al dataset](https://www.kaggle.com/datasets/nikhil7280/weather-type-classification/)
* es una modificacion del dataset original.



Estamos frente a un dataset que tiene 10.090 entradas y 8 columnas con tipos de datos int, float y object.

In [150]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10090 entries, 0 to 10089
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Temperatura         10090 non-null  int64  
 1   Humedad             10090 non-null  int64  
 2   VientoVelocidad     10090 non-null  float64
 3   Precipitation       10090 non-null  int64  
 4   PresionAtmosferica  10090 non-null  float64
 5   Localizacion        10090 non-null  object 
 6   TipoClima           10090 non-null  object 
 7   Estacion            10090 non-null  object 
dtypes: float64(2), int64(3), object(3)
memory usage: 630.8+ KB


La columna 'Precipitation' está en inglés, mientras que las demás están en español.

In [151]:
df.columns

Index(['Temperatura', 'Humedad', 'VientoVelocidad', 'Precipitation',
       'PresionAtmosferica', 'Localizacion', 'TipoClima', 'Estacion'],
      dtype='object')

Creo una copia para trabajar el dataset sin sobreescribir el original.

In [152]:
df_renamed: pd.DataFrame = df.copy()

In [153]:
df_renamed['Precipitacion'] = df_renamed['Precipitation']
df_renamed = df_renamed.drop(columns=['Precipitation'])

In [154]:
df_renamed.columns

Index(['Temperatura', 'Humedad', 'VientoVelocidad', 'PresionAtmosferica',
       'Localizacion', 'TipoClima', 'Estacion', 'Precipitacion'],
      dtype='object')

Exploración de una muestra chica del dataset.  

In [155]:
df_renamed.sample(5)

,Temperatura,Humedad,VientoVelocidad,PresionAtmosferica,Localizacion,TipoClima,Estacion,Precipitacion
2670,32,31,12.0,1063.71,Costa,Soleado,Otonio,29
114,-5,68,0.5,992.19,Montania,Nevado,Invierno,96
556,3,97,3.0,991.12,Llanura,Nevado,Invierno,66
5697,-2,99,17.0,990.96,Llanura,Nevado,Invierno,96
911,44,67,1.5,1021.24,Costa,Soleado,Otonio,8


Chequeo valores únicos en las columnas categóricas. De esta manera descarto la existencia de la letra 'ñ' en el caso de Otonio y Montania y que todos los campos incian con mayúsculas.

In [156]:
print(pd.unique(df_renamed['Estacion']))
print(pd.unique(df_renamed['Localizacion']))
print(pd.unique(df_renamed['TipoClima']))

['Primavera' 'Verano' 'Invierno' 'Otonio']
['Llanura' 'Montania' 'Costa']
['Nublado' 'Soleado' 'Nevado' 'Lluvioso']


Chequeo de valores faltantes: no existen faltantes en el dataset.

In [157]:
df_renamed.isna().sum()

Temperatura           0
Humedad               0
VientoVelocidad       0
PresionAtmosferica    0
Localizacion          0
TipoClima             0
Estacion              0
Precipitacion         0
dtype: int64

In [158]:
columnas_categoricas: list[str] = ['Localizacion', 'TipoClima', 'Estacion']
columnas_numericas: list[str] = ['Temperatura', 'Humedad', 'VientoVelocidad','PresionAtmosferica','Precipitacion']

Vemos que tenemos un gran desvío estandar en las variables Precipitacion, Temperatura y PresionAtmosferica

In [159]:
df_renamed.describe()

,Temperatura,Humedad,VientoVelocidad,PresionAtmosferica,Precipitacion
count,10090.000000,10090.000000,10090.000000,10090.000000,10090.000000
mean,21.872349,67.195045,9.634936,1007.685836,50.796333
std,16.426722,20.463797,6.795890,38.763971,32.333798
min,-24.000000,20.000000,0.000000,800.230000,0.000000
25%,13.000000,55.000000,5.000000,998.570000,18.000000
50%,23.000000,69.000000,8.500000,1010.315000,53.000000
75%,32.000000,81.000000,13.500000,1017.820000,80.000000
max,109.000000,109.000000,47.500000,1199.210000,109.000000


Debido a las distintas escalas de los datos, más allá que vemos necesario normalizar, se procede a visualizar los datos con un histograma.

In [160]:
df_renamed.columns

Index(['Temperatura', 'Humedad', 'VientoVelocidad', 'PresionAtmosferica',
       'Localizacion', 'TipoClima', 'Estacion', 'Precipitacion'],
      dtype='object')

In [161]:
histogram_ = px.histogram(df_renamed.drop(columns=['Estacion','TipoClima','Localizacion']), title='Histograma de los datos Temperatura, Humedad, Velocidad de viento y Presion Atmosferica', nbins=500)
histogram_.show()

Distribuciones observadas:
- Temperatura, Humedad, VientoVelocidad PresionAtmosferica tienen una distribucion normal. PresionAtmosferica tiene una colas pesadas tanto hacia la derecha como hacia la izquierda. VientoVelocidad presenta una cola pesada hacia la derecha. 
- Precipitacion tiene a ser uniforme.

Análisis de outliers



In [162]:
px.box(df_renamed[columnas_numericas], width=1000, height=2000)

In [163]:
px.scatter(df_renamed['Temperatura'], color=df_renamed['Estacion'], title='Temperatura diferenciada por estación') 

In [164]:
px.scatter(df_renamed['VientoVelocidad'], color=df_renamed['Estacion'], title='VientoVelocidad en cada Estacion')

En ambos scatterplots, podemos observar que las temperaturas no tienen relacion con las estaciones del anio. Esto se debe a que es un dataset generado de forma sintetica. En el caso del viento, se visualiza algo similar.

Procedemos a realizar la matriz de correlación para ver cual es la relación entre las variables.

In [165]:
corr = df_renamed[columnas_numericas].corr()

px.imshow(corr,  zmin=-1, zmax=1, title='Correlación de las variables', text_auto=True, color_continuous_scale='RdBu', aspect='auto')

No vemos correlaciones fuertes. Sin embargo, podemos destacar humedad - precipitacion. Esto es logico.

Teniendo en cuenta los analisis inmediatamente anteriores, se toma como criterio que los valores por debajo del quantil 1 y por encima del quantil 99, son outliers. Como la mayoría son distribuciones normales, optamos por rellenar esos outliers con la mediana.

In [166]:
df_outliers: pd.DataFrame = pd.DataFrame()
df_without_outliers: pd.DataFrame = df_renamed.copy()

for column in columnas_numericas:
    q1 = df_renamed[column].quantile(0.01)
    q99 = df_renamed[column].quantile(0.99)
    median_ = df_renamed[column].median()

    outliers: pd.DataFrame = df_renamed.loc[
        (df_renamed[column] < q1) | (df_renamed[column] > q99)
        ]

    df_outliers = pd.concat([df_outliers, outliers])
    df_without_outliers[column] = df_without_outliers[column].apply(lambda x: median_ if x < q1 or x > q99 else x)


df_outliers = df_outliers.drop_duplicates()

print(f'Existían {df_outliers.shape[0]} filas con outliers.')
print(f'Los outliers representaban un {100*df_outliers.shape[0]/df_renamed.shape[0]:.2f}% de la población.')


Existían 655 filas con outliers.
Los outliers representaban un 6.49% de la población.


## Estandarización

Se utiliza la técnica Z-Score.

In [167]:
scaler: StandardScaler = StandardScaler()

df_scaled: pd.DataFrame = pd.DataFrame(scaler.fit_transform(df_without_outliers.drop(columns=columnas_categoricas)), columns=columnas_numericas)

In [168]:
df_scaled = pd.concat([df_scaled, df_without_outliers[columnas_categoricas]], axis=1)
df_scaled.head()

,Temperatura,Humedad,VientoVelocidad,PresionAtmosferica,Precipitacion,Localizacion,TipoClima,Estacion
0,1.161284,1.448314,-0.137236,0.122036,0.651490,Llanura,Nublado,Primavera
1,0.558133,-0.162720,-0.381812,0.373370,-1.076321,Montania,Soleado,Primavera
2,1.094267,0.793832,-1.278592,0.632978,0.997053,Costa,Soleado,Primavera
3,0.692166,-0.615823,-0.952490,0.073769,-0.762174,Llanura,Nublado,Verano
4,-1.251321,0.894521,-0.544863,-0.807796,1.436860,Llanura,Nevado,Invierno


Chequeamos que la estandarización de los datos es correcta.

In [169]:
df_scaled.describe()

,Temperatura,Humedad,VientoVelocidad,PresionAtmosferica,Precipitacion
count,1.009000e+04,1.009000e+04,1.009000e+04,1.009000e+04,1.009000e+04
mean,-5.545614e-17,-1.028139e-16,1.690092e-17,2.851678e-15,-3.943547e-17
std,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00,1.000050e+00
min,-2.524641e+00,-2.327547e+00,-1.523169e+00,-5.804133e+00,-1.578958e+00
25%,-5.811533e-01,-6.158235e-01,-7.079140e-01,-2.978881e-01,-1.013492e+00
50%,8.901480e-02,8.900406e-02,-1.372359e-01,8.359460e-02,8.602477e-02
75%,6.251493e-01,6.931419e-01,5.964932e-01,3.306189e-01,9.028086e-01
max,3.104771e+00,2.002107e+00,4.102088e+00,5.370397e+00,1.751007e+00




Chequeo de balanceo de clases target: vemos que las clases están prácticamente balanceadas.


In [170]:
df_scaled['Estacion'].value_counts()

Estacion
Primavera    2598
Invierno     2500
Otonio       2500
Verano       2492
Name: count, dtype: int64

In [171]:
df_dummies: pd.DataFrame = pd.get_dummies(df_scaled, columns=['Localizacion', 'TipoClima'], dtype=int)
df_dummies

,Temperatura,Humedad,VientoVelocidad,PresionAtmosferica,Precipitacion,Estacion,Localizacion_Costa,Localizacion_Llanura,Localizacion_Montania,TipoClima_Lluvioso,TipoClima_Nevado,TipoClima_Nublado,TipoClima_Soleado
0,1.161284,1.448314,-0.137236,0.122036,0.651490,Primavera,0,1,0,0,0,1,0
1,0.558133,-0.162720,-0.381812,0.373370,-1.076321,Primavera,0,0,1,0,0,0,1
2,1.094267,0.793832,-1.278592,0.632978,0.997053,Primavera,1,0,0,0,0,0,1
3,0.692166,-0.615823,-0.952490,0.073769,-0.762174,Verano,0,1,0,0,0,1,0
4,-1.251321,0.894521,-0.544863,-0.807796,1.436860,Invierno,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10085,0.625149,-2.176513,-0.218761,0.748819,-1.421884,Verano,0,1,0,0,0,0,1
10086,0.491116,-0.263410,0.596493,-0.175152,-1.044907,Primavera,1,0,0,0,0,1,0
10087,-0.782204,0.340728,0.841070,-0.163430,0.651490,Verano,0,0,1,1,0,0,0
10088,0.558133,0.491763,-0.626389,0.165476,-0.699344,Otonio,1,0,0,0,0,1,0


## Implementacion de Modelos

### Separación de dataset en tran y test




Primeramente, se selecciona la variable target 'Estacion' y se crea también el dataset X con las variables restantes.


In [172]:
X: pd.DataFrame = df_dummies.drop(columns=['Estacion'])
y: pd.DataFrame = df_dummies[['Estacion']]



Separación en Train y Test. Se utiliza un 20% de para los datos de test y un 80% para los de train.


In [173]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### SVC: Maquina de Vectores Soporte. 

#### Linear Kernel



Definición de un diccionario que contiene los posibles valores de C (costo).


In [174]:
parametros: Dict[str, List[float]] = {
    'C' : [0.1,0.05 ,0.2 ,0.5 ,0.8, 1 , 5, 10 , 40]
}

Se inicializa el modelo de SVM y se busca el mejor mediante GridSearch. Se utiliza 'recall' como métrica ya que es importante que diferencie bien entre categorías. No optimizamos gamma ya que no participa en el kernel lineal. 

[Link Documentación de SVM, inciso 1.4.6](https://scikit-learn.org/stable/modules/svm.html)

In [178]:
recall_parameter: dict[float, float] = {}

for c in parametros['C']:
    svc_linear = SVC(C=c, kernel='linear', random_state=42)
    svc_linear.fit(X_train, y_train)
    y_pred_svc_linear = svc_linear.predict(X_test
                                           )
    recall_parameter[c] = recall_score(y_test, y_pred_svc_linear, average='weighted')

    print(f'C={c}: \n')
    print(classification_report(y_test, y_pred_svc_linear))
    print("-------------")


C=0.1: 

              precision    recall  f1-score   support

    Invierno       0.86      0.54      0.67       498
      Otonio       0.28      0.16      0.21       506
   Primavera       0.30      0.62      0.41       526
      Verano       0.30      0.20      0.25       488

    accuracy                           0.39      2018
   macro avg       0.44      0.38      0.38      2018
weighted avg       0.44      0.39      0.38      2018

-------------
C=0.05: 

              precision    recall  f1-score   support

    Invierno       0.86      0.54      0.67       498
      Otonio       0.27      0.15      0.19       506
   Primavera       0.31      0.67      0.42       526
      Verano       0.30      0.17      0.22       488

    accuracy                           0.39      2018
   macro avg       0.44      0.38      0.37      2018
weighted avg       0.43      0.39      0.37      2018

-------------
C=0.2: 

              precision    recall  f1-score   support

    Invierno       

Se observa que a partir de un C = 0.2, se mantienen las métricas. Por ende, elegimos ese. 

Cross - validation:

In [185]:
svc_linear_kfold = SVC(C=0.2, kernel='linear', random_state=42)

In [192]:
sfolder: StratifiedKFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

recall_metrics: Dict[str, float] = {}
f1_metrics: Dict[str, float] = {}

gen = sfolder.split(X, y)
print(type(prueba))

print(prueba)

for train_index, test_index in prueba:
    
    x_train_fold, x_test_fold = X.iloc[train_index], X.iloc[test_index]  
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]

    svc_linear_kfold.fit(x_train_fold, y_train_fold)

    y_pred_svc_linear_kfold = svc_linear_kfold.predict(x_test_fold)
    
    recall_metrics[f'C={svc_linear_kfold.C}'] = recall_score(y_test_fold, y_pred_svc_linear_kfold, average='macro')
    f1_metrics[f'C={svc_linear_kfold.C}'] = f1_score(y_test_fold, y_pred_svc_linear_kfold, average='macro')

<class 'generator'>
<generator object _BaseKFold.split at 0x788eb1f1abd0>


In [193]:
print(recall_metrics)
f1_metrics

{'C=0.2': np.float64(0.3785565350353887)}


{'C=0.2': np.float64(0.3673482575336259)}